In [ ]:
import os
import json
import os
import numpy as np

import cv2
import os
import numpy as np
import albumentations as A

In [ ]:
transform = A.Compose(
    [
        A.CLAHE(),
        A.RandomRotate90(),
        A.Transpose(),
        A.ShiftScaleRotate(
            shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=0.75
        ),
        A.Blur(blur_limit=3),
        A.OpticalDistortion(),
        A.GridDistortion(),
        A.HueSaturationValue(),
    ],
    bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]),
)

In [ ]:
transform[1]

In [ ]:
imagesTransformed = []
from matplotlib import pyplot as plt


def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis("off")
    plt.imshow(image)

In [ ]:
def generate_bounding_boxes_data(data_folder):
    """
    Generate bounding boxes data from the files in the data folder for fine-tuning YOLOv7.
    Args:
      data_folder: The path to the folder containing the data.
    """
    # Define file patterns
    bounding_box_labels_pattern = "bounding_box_2d_tight_labels_*.json"
    bounding_boxes_pattern = "bounding_box_2d_tight_*.npy"
    # Get a list of bounding box label files
    bounding_box_label_files = [
        f
        for f in os.listdir(data_folder)
        if f.startswith("bounding_box_2d_tight_labels_")
    ]
    bounding_boxes_data = []
    for label_file in bounding_box_label_files:
        number = label_file.split("_")[-1].split(".")[0]
        bounding_box_file = f"bounding_box_2d_tight_{number}.npy"
        with open(os.path.join(data_folder, label_file), "r") as f:
            label_data = json.load(f)
        bounding_box_data = {
            "image_file": f"rgb_{number}.png",
            "bounding_box_labels": label_data,
            "bounding_box_data": np.load(os.path.join(data_folder, bounding_box_file)),
        }
        bounding_boxes_data.append(bounding_box_data)
    return bounding_boxes_data


# Example usage
data_folder = "../data"
bounding_boxes_data = generate_bounding_boxes_data(data_folder)

In [ ]:
bounding_boxes_data[0]

In [ ]:
# ... existing imports ...
transform = A.Compose(
    [
        A.CLAHE(),
        A.RandomRotate90(),
        A.Transpose(),
        A.ShiftScaleRotate(
            shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=0.75
        ),
        A.Blur(blur_limit=3),
        A.OpticalDistortion(),
        A.GridDistortion(),
        A.HueSaturationValue(),
    ],
    bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]),
)


def apply_transformations(bounding_boxes_data, transform, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for idx, data in enumerate(bounding_boxes_data):
        try:
            img_file = data["image_file"]
            img_path = os.path.join("./data", img_file)
            img = cv2.imread(img_path)

            if img is None:
                print(f"Failed to read image: {img_path}")
                continue

            bboxes = data["bounding_box_data"]

            class_labels = [int(x[0]) for x in bboxes]

            # Convert bboxes to YOLO format
            height, width = img.shape[:2]
            yolo_bboxes = []
            for bbox in bboxes:
                x_min, y_min, x_max, y_max = bbox[1], bbox[2], bbox[3], bbox[4]
                x_center = (x_min + x_max) / (2 * width)
                y_center = (y_min + y_max) / (2 * height)
                box_width = (x_max - x_min) / width
                box_height = (y_max - y_min) / height
                yolo_bboxes.append([x_center, y_center, box_width, box_height])

            # Apply all transformations
            for trans_idx, trans in enumerate(transform):
                transformed = trans(
                    image=img, bboxes=yolo_bboxes, class_labels=class_labels
                )
                transformed_image = transformed["image"]
                transformed_bboxes = transformed["bboxes"]
                transformed_labels = transformed["class_labels"]

                # Save transformed image
                output_image_file = f"{os.path.splitext(img_file)[0]}_{trans_idx}{os.path.splitext(img_file)[1]}"
                output_path = os.path.join(output_folder, output_image_file)
                cv2.imwrite(output_path, transformed_image)

                # Save transformed labels
                label_file = f"{os.path.splitext(img_file)[0]}_{trans_idx}.txt"
                label_path = os.path.join(output_folder, label_file)

                with open(label_path, "w") as file:
                    for label, bbox in zip(transformed_labels, transformed_bboxes):
                        file.write(f"{label} {' '.join(map(str, bbox))}\n")

            print(f"Processed image {idx + 1}/{len(bounding_boxes_data)}: {img_file}")

        except Exception as e:
            print(f"Error processing {img_file}: {str(e)}")
            continue


# Usage
output_folder = "./transformedData"
apply_transformations(bounding_boxes_data, transform, output_folder)

In [ ]:
def draw_and_plot_yolo_boxes(image, boxes):
    height, width, _ = image.shape
    for box in boxes:
        x_center, y_center, box_width, box_height = box
        x_center *= width
        y_center *= height
        box_width *= width
        box_height *= height
        
        x1 = int(x_center - (box_width / 2))
        y1 = int(y_center - (box_height / 2))
        x2 = int(x_center + (box_width / 2))
        y2 = int(y_center + (box_height / 2))
        
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Convert BGR image to RGB for displaying with matplotlib
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Plot the image
    plt.figure(figsize=(10, 10))
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()

draw_and_plot_yolo_boxes(transformed_image, transformed_bboxes)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
def read_transformed_bounding_boxes_data(boxesPath):
    transformed_bounding_boxes_data = []

    with open(boxesPath, "rb") as f:
        lines = f.readlines()
        bounding_boxes = []
        for line in lines:

            try:
                parts = line.strip().split()
                label = int(float(parts[0]))
                bbox = list(map(float, parts[1:]))
                bounding_boxes.append([label, bbox])
            except Exception as e:
                print(boxesPath, e)
                return
        transformed_bounding_boxes_data.append(bounding_boxes)

    return transformed_bounding_boxes_data


### Function to display images with bounding boxes
def displayImagesWithBoxesYolo(image_path, boxesPath):
    """
    Plots bounding boxes on an image.

    Args:
      image_path: Path to the image file.
      boxesPath: Path to the bounding boxes file.
    """

    # Load the image
    image = Image.open(image_path)
    bounding_boxes = read_transformed_bounding_boxes_data(boxesPath)
    bounding_boxes = [x[1] for x in bounding_boxes[0]]

    plt.imshow(image)

    # Convert bounding boxes to xmin, ymin, xmax, ymax
    for x_center, y_center, width, height in bounding_boxes:
        xmin = int((x_center - width / 2) * image.size[0])
        ymin = int((y_center - height / 2) * image.size[1])
        xmax = int((x_center + width / 2) * image.size[0])
        ymax = int((y_center + height / 2) * image.size[1])

        # Plot the bounding box
        rect = plt.Rectangle(
            (xmin, ymin),
            xmax - xmin,
            ymax - ymin,
            fill=False,
            edgecolor="red",
            linewidth=2,
        )
        plt.gca().add_patch(rect)

    plt.axis("off")
    plt.show()

In [ ]:
displayImagesWithBoxesYolo(
    "./transformedData/rgb_0000_0.png", "./transformedData/rgb_0000_0.txt"
)

In [ ]:
displayImagesWithBoxesYolo(
    "./yolov7/yolov7_data/test/images/rgb_0010.png",
    "./yolov7/yolov7_data/test/labels/rgb_0010.txt",
)